In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gekko import GEKKO
from tabulate import tabulate

In [11]:
df = pd.read_csv('DataSet.csv')
df

Subset1_BS_S001_loyalonly (1) copy
Customer_Segment Store_ID Product/Bundle_ID                               Promo_Flag Num_Units_X Num_Units_Y Regular_Price      Promo_Price        Discount_Percentage Total_Sales Price_Elasticity    Product_Description                             Predicted_Sales X          Y      Reg_X              Reg_Y              cost_X             cost_Y                                      unit_cost
Bargain Seekers  S001     Whole Wheat Loaf                                1.0        NaN         NaN         26.57692307692310  23.421538461538500 0.1203633009001040  21          -1.8511456744814700 Whole Wheat Loaf                                22              NaN        NaN    NaN                NaN                NaN                NaN                                               2.5
                          Bundle of 4 Organic Eggs + 4 Oatmeal\nPacks     1.0        4.0         4.0         56.1               37.0               0.3404329070578740  8           -1.893073352605490  Bundle of 4 Organic Eggs + 4 Oatmeal\nPacks     9               Eggs       Cereal 27.19610015451610  27.006694605009600 6.79902503862902   6.751673651252410                              14.025
                          Bundle of 2 Cheddar Blocks + 2 Gra-\nnola Boxes 1.0        2.0         2.0         16.89              12.84              0.2399544277988750  7           -2.157574808323880  Bundle of 2 Cheddar Blocks + 2 Gra-\nnola Boxes 8               Cheese     Cereal 27.48519622093020  27.006694605009600 6.871299055232560  6.751673651252410                              4.2225
                          Bundle of 3 Salted Butter + 4 Greek\nYogurts    1.0        3.0         4.0         43.26              34.61              0.19984478118673    11          -2.058769318473920  Bundle of 3 Salted Butter + 4 Greek\nYogurts    12              Butter     Yogurt 27.061996129656500 28.44293571082000  6.765499032414130  7.110733927705000                              10.815
                          Bundle of 4 Herb Butter + 1 Baguette            1.0        4.0         1.0         23.88              17.12              0.2830499918274970  3           -2.3312787993483000 Bundle of 4 Herb Butter + 1 Baguette            4               Butter     Bread  27.061996129656500 26.721874385447400 6.765499032414130  6.680468596361850                                5.97
                          Bundle of 5 Orange Juices + 5 Almond\nMilks     1.0        5.0         5.0         46.84              33.36              0.2877040546867110  8           -2.1779260335285200 Bundle of 5 Orange Juices + 5 Almond\nMilks     9               Juice      Milk   27.31916881874700  27.523031473533600 6.829792204686750  6.880757868383400                               11.71
                          Bundle of 3 Cornflakes + 2 Soy Milks            1.0        3.0         2.0         71.88              64.26              0.1060645475176490  7           -1.9276175761958300 Bundle of 3 Cornflakes + 2 Soy Milks            8               Cereal     Milk   27.006694605009600 27.523031473533600 6.751673651252410  6.880757868383400                               17.97
                          Bundle of 3 Vanilla Yogurts + 3 Ghee\nTubs      1.0        3.0         3.0         97.56              85.57              0.1229324462074060  3           -1.9983214832057000 Bundle of 3 Vanilla Yogurts + 3 Ghee\nTubs      4               Yogurt     Butter 28.44293571082000  27.061996129656500 7.110733927705000  6.765499032414130                               24.39
                          Bundle of 5 Mixed Vegetables + 2 Fresh\nBerries 1.0        5.0         2.0         71.59              38.53              0.4617816936356140  3           -1.9994077311300000 Bundle of 5 Mixed Vegetables + 2 Fresh\nBerries 4               Vegetables Fruit  27.573760049019600 27.510738907729300 6.893440012254900  6.8776847269323200                            17.8975
                          Bundle of 1 Garlic 

In [19]:
# Main function to solve the MINLP pricing optimization problem
def Solve_MINLP(df, WarmStart_delta, WarmStart_d, TotalBudget, RevenueTarget, MarginTarget,
                       MaxDiscount, MinDiscount, MaxSKU, MaxBundles, MaxCons, w3, w4):
    # Initialize GEKKO model with local (offline) solving
    m = GEKKO(remote=False)
    m.options.SOLVER = 1  # Use APOPT solver (good for MINLP problems)

    # optional solver settings with APOPT
    m.solver_options = ['minlp_maximum_iterations 2000', \
                        # minlp iterations with integer solution
                        'minlp_max_iter_with_int_sol 10', \
                        # treat minlp as nlp
                        'minlp_as_nlp 0', \
                        # nlp sub-problem max iterations
                        'nlp_maximum_iterations 100', \
                        # 1 = depth first, 2 = breadth first
                        'minlp_branch_method 1', \
                        # maximum deviation from whole number
                        'minlp_integer_tol 0.05', \
                        # covergence tolerance
                        'minlp_gap_tol 0.01']
 
    # Compute weights for revenue and margin to be used in objective function
    total_target = RevenueTarget + MarginTarget
    w1 = RevenueTarget / total_target
    w2 = MarginTarget / total_target
 
    # Initialize constants
    n_items = len(df)  # Number of SKUs
 
    # Declare decision variables:
    # Binary selection variable δ_i (1 if selected, 0 otherwise)
    delta = [m.Var(value=WarmStart_delta[i], lb=0, ub=1, integer=True) for i in range(n_items)]
    # Discount variable d_i (fractional discount between 0 and 1)
    d = [m.Var(value=WarmStart_d[i], lb=0, ub=1) for i in range(n_items)]
    # Revenue and margin variables (continuous, ≥ 0)
    r = [m.Var(lb=0) for _ in range(n_items)]
    m_i = [m.Var(lb=0) for _ in range(n_items)]
 
    # Slack variables for deviation from revenue/margin targets
    z_R = m.Var(lb=0)
    z_M = m.Var(lb=0)
 
    # Extract pricing, sales, elasticity, and cost data from dataframe
    P0 = df['Regular_Price'].values
    S0 = df['Total_Sales'].values
    PE = df['Price_Elasticity'].values
    cost = df['unit_cost'].values

    # Define constraints and intermediate equations per SKU
    for i in range(n_items):
        # Discounted price formula using binary and discount variables
        P = P0[i] * (1 - delta[i] * d[i])
        # Demand function using own-price elasticity (log-log form)
        S = S0[i] * ((P / P0[i]) ** PE[i])
        # Revenue = Price × Demand
        m.Equation(r[i] == S * P)
        # Margin = Demand × (Price - Cost)
        m.Equation(m_i[i] == S * (P - cost[i]))
 
        # Constraint: Discounted price must fall within allowed min and max bounds
        m.Equation(P >= P0[i] * (1 - MaxDiscount[i]))
        m.Equation(P <= P0[i] * (1 - MinDiscount[i]))


    # Constraints
    m.Equation(m.sum([delta[i] for i in range(n_items)]) <= MaxSKU)
    m.Equation(m.sum(m_i) >= MarginTarget - z_M)
    m.Equation(m.sum(r) >= RevenueTarget - z_R)
    m.Equation(z_R >= 0)
    m.Equation(z_M >= 0)

    m.Equation(m.sum([delta[i] * P0[i] * d[i] * (S0[i] * ((P / P0[i]) ** PE[i])) for i in range(len(df))]) <= TotalBudget)

    # Objective
    obj = w1 * m.sum(r) + w2 * m.sum(m_i) - w3 * z_R - w4 * z_M
    m.Obj(-obj)
    
    try:
        m.solve(disp=True)
    except Exception as e:
        print(" GEKKO failed. Opening model folder for diagnosis...")
        m.open_folder()
        raise e
    return {
        'delta': [d.value[0] for d in delta],
        'd': [dd.value[0] for dd in d],
        'r': [rr.value[0] for rr in r],
        'margin': [mm.value[0] for mm in m_i],
        'z_R': z_R.value[0],
        'z_M': z_M.value[0]
    }


def print_results(results, df):
    delta = results['delta']
    discounts = results['d']
    revenues = results['r']
    margins = results['margin']
    product_names = df['Product_Description'].values
    
    summary_df = pd.DataFrame({'Product Name ': product_names,'Selected ': ['Selected' if int(d) == 1 else 'Not Selected' for d in delta],\
                               'Discount % ': [f"{round(dis * 100, 2)}%" if sel >= 0.5 else "0.0%" for dis, sel in zip(discounts, delta)],\
                               'Revenue ': [f"${round(r, 2)}" for r in revenues],'Margin ': [f"${round(m, 2)}" for m in margins]})
    
    print("\n OPTIMIZATION RESULTS: ITEM-LEVEL OVERVIEW")
    print(tabulate(summary_df, headers='keys', tablefmt='fancy_grid', showindex=False))
    print("\n\033[1;32m Summary Metrics:\033[0m")
    print(f" Total Selected Items: {sum(1 for d in delta if d >= 0.5)}")
    print(f" Total Revenue: \033[1m${round(sum(revenues), 2)}\033[0m")
    #print(f" Total Margin: \033[1m${round(sum(margins), 2)}\033[0m")


if __name__ == "__main__":
    # Load data (make sure this file exists and has the correct columns)
    df = pd.read_csv('DataSet - dataset.csv')
 
    # Initialize model inputs
    n = len(df)
    WarmStart_delta = [0] * n           # Start with no items selected
    WarmStart_d = [0.01] * n             # Start with 1% discount across items
    MaxDiscount = np.minimum(1,df['Discount_Percentage'] + 10/100)             # Max discount = 30%
    MinDiscount = [0.0] * n            # Min discount = 0%
    MaxSKU = 17                         # Max SKUs allowed to discount
    MaxBundles = 0                      # Bundle logic not implemented yet
    MaxCons = 0                         # Constraint groups not implemented yet
    RevenueTarget = 20e3                # Minimum revenue desired
    MarginTarget = 10e3                 # Minimum margin desired
    TotalBudget = 10e3
 
    w3 = 10000                          # Penalty for not hitting revenue target
    w4 = 10000                           # Penalty for not hitting margin target
 
    # Run optimization and print results
    results = Solve_MINLP(df, WarmStart_delta, WarmStart_d, TotalBudget, RevenueTarget,
                                  MarginTarget, MaxDiscount, MinDiscount, MaxSKU, MaxBundles,
                                  MaxCons, w3, w4)
    print_results(results, df)

 ----------------------------------------------------------------
 APMonitor, Version 1.0.3
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            6
   Constants    :            0
   Variables    :          154
   Intermediates:            0
   Connections  :          126
   Equations    :          107
   Residuals    :          107
 
 Number of state variables:            154
 Number of total equations: -          112
 Number of slack variables: -           46
 ---------------------------------------
 Degrees of freedom       :             -4
 
 * Warning: DOF <= 0
 ----------------------------------------------
 Steady State Optimization with APOPT Solver
 ----------------------------------------------
Iter:     1 I:  0 Tm:      0.01 NLPi:    7 Dpth:    0 Lvs:    3 Obj:  4.43E+07 Gap:       NaN
--Integer Solution:   4.74E+07 Lowest Leaf:   4.43E+07